In [ ]:
# default_exp environment

# Environment

> Environment/Callback

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export

from mrl.imports import *
from mrl.torch_imports import *
from mrl.torch_core import *
from mrl.chem import *
from mrl.templates import *
from mrl.agent import *

//anaconda3/envs/mrl/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)


In [ ]:
# export

class Callback():
    def __init__(self, name='callback', order=10):
        self.order=order
        self.name = name
    
    def __call__(self, event_name):
        
        event = getattr(self, event_name, None)
        if event is not None:
            output = event()
        else:
            output = None
            
        return output
    
class Log(Callback):
    def __init__(self):
        super().__init__(name='log', order=100)
        
        self.pbar = None
        self.iterations = 0
        self.metrics = {}
        self.metrics['rewards']
        
        self.log = {}
        self.log['samples']
        self.log['sources']
        self.log['rewards']
        
        self.timelog = {}
        
        self.report = 1
        self.do_log = True
        self.unique_samples = set()
        
    def before_train(self):
        cols = ['iterations'] + list(self.metrics.keys())
        if self.pbar is None:
            print('\t'.join(cols))
        else:
            self.pbar.write(cols, table=True)
            
    def add_metric(self, name):
        self.metrics[name]
        
    def add_log(self, name):
        self.log[name]
        
    def update_metric(self, name, value):
        if self.do_log:
            self.metrics[name].append(value)
        
    def update_log(self):
        if self.do_log:
            env = self.environment
            batch_state = env.batch_state
            samples = batch_state.samples
            self.unique_samples.update(set(samples))

            for key in self.log.keys():
                items = batch_state[key]
                if isinstance(items, torch.Tensor):
                    items = items.detach().cpu().numpy()
                self.log[key].append(items)
        
    def report_batch(self):
        outputs = [f'{self.iterations}']
        if self.iterations%self.report==0:
            
            for k,v in self.metrics.items():
                val = v[-1]

                if type(val)==int:
                    val = f'{val}'
                else:
                    val = f'{val:.3f}'

                outputs.append(val)

            if self.pbar is None:
                print('\t'.join(outputs))
            else:
                self.pbar.write(outputs, table=True)
            
        self.iterations += 1
        
    def after_batch(self):
        self.update_log()
        self.report_batch()

    
class Buffer(Callback):
    def __init__(self, p_total, max_size=1000000):
        super().__init__(name='buffer', order=0)
        
        self.buffer = []
        self.used_buffer = []
        self.max_size = max_size
        self.p_total = p_total
        self.buffer_valid = []
        
    def __len__(self):
        return len(self.buffer)
    
    def add(self, item):
        
        if is_container(item):
            for i in item:
                self.add(i)
        else:
            place_idx = (len(self.buffer)%self.max_size)+1

            if place_idx>=len(self.buffer):
                self.buffer.append(item)
            else:
                self.buffer[place_idx-1] = item
                
    def setup(self):
        log = self.environment.log
        log.add_metric(f'diversity')
        log.add_metric(f'valid')
        log.add_metric(f'bs')
            
    def sample(self, n):
        
        idxs = np.random.choice(np.arange(len(self.buffer)), min(n, len(self.buffer)), 
                                replace=False)
        batch = [self.buffer[i] for i in idxs]
        for idx in sorted(idxs, reverse=True):
            self.buffer.pop(idx)

        return batch
    
    def after_sample(self):
        template_cb = self.environment.template_cb
        samples = self.batch_state.samples
        sources = np.array(self.batch_state.sources)
        samples = template_cb.standardize(samples)
        valids = template_cb.filter_sequences(samples, return_array=True)
        
        if valids.mean()<1.:
            filtered_samples = [samples[i] for i in range(len(samples)) if valids[i]]
            filtered_sources = [sources[i] for i in range(len(sources)) if valids[i]]
            filtered_latent_data = {}

            for source,latent_idxs in self.batch_state.latent_data.items():
                valid_subset = valids[sources==source]
                latent_filtered = latent_idxs[valid_subset]
                filtered_latent_data[source] = latent_filtered

            self.batch_state.samples = filtered_samples
            self.batch_state.sources = filtered_sources
            self.batch_state.latent_data = filtered_latent_data
            
        self.used_buffer += samples
        
        diversity = len(set(self.batch_state.samples))/len(self.batch_state.samples)
        self.environment.log.update_metric('diversity', diversity)
        self.environment.log.update_metric('valid', valids.mean())
        self.environment.log.update_metric('bs', len(self.batch_state.samples))
        
        if self.environment.log.iterations%40 == 0:
            self.used_buffer = list(set(self.used_buffer))
            
            if len(self.used_buffer)>self.max_size:
                self.used_buffer = self.used_buffer[-self.max_size:]
        
    def after_build_buffer(self):
        template_cb = self.environment.template_cb
        if self.buffer:
            len1 = len(self.buffer)
            self.buffer = template_cb.standardize(self.buffer)
            self.buffer = list(set(self.buffer))
            self.buffer = template_cb.filter_sequences(self.buffer)
            self.buffer_valid.append(len(self.buffer)/len1)
    
    def sample_batch(self):
        bs = int(self.environment.bs * self.p_total)
        if bs>0:
            sample = self.sample(bs)
            self.batch_state.samples += sample
            self.batch_state.sources += ['buffer']*len(sample)

In [ ]:
# export

class SettrDict(dict):
    def __init__(self):
        super().__init__()
        
    def __setitem__(self, key, item):
        super().__setitem__(key, item)
        super().__setattr__(key, item)
    
    def __setattr__(self, key, item):
        super().__setitem__(key, item)
        super().__setattr__(key, item)
        
    def update_from_dict(self, update_dict):
        for k,v in update_dict.items():
            self[k] = v
        
class BatchState(SettrDict):
    def __init__(self):
        super().__init__()
        
        self.samples = []
        self.sources = []
        self.rewards = to_device(torch.tensor(0.))
        self.loss = to_device(torch.tensor(0.))
        self.latent_data = {}


In [ ]:
# export

class Event():
    def __init__(self):
        self.setup = 'setup'
        self.before_train = 'before_train'
        self.build_buffer = 'build_buffer'
        self.after_build_buffer = 'after_build_buffer'
        self.before_batch = 'before_batch'
        self.sample_batch = 'sample_batch'
        self.after_sample = 'after_sample'
        self.get_model_outputs = 'get_model_outputs'
        self.compute_reward = 'compute_reward'
        self.after_compute_reward = 'after_compute_reward'
        self.compute_loss = 'compute_loss'
        self.zero_grad = 'zero_grad'
        self.before_step = 'before_step'
        self.step = 'step'
        self.after_batch = 'after_batch'
        self.after_train = 'after_train'

In [ ]:
# export

class Environment():
    def __init__(self, agent_cb, template_cb=None, samplers=[], reward_cbs=[], loss_cbs=[], cbs=[],
                buffer_p_batch=None, reward_decay=0.9):
        self.agent_cb = agent_cb
        self.template_cb = template_cb if template_cb is not None else TemplateCallback()
        self.samplers = samplers
        self.reward_cbs = reward_cbs
        self.loss_cbs = loss_cbs
        self.cbs = []
        if buffer_p_batch is None:
            buffer_p_batch = 1.
            for samp in samplers:
                buffer_p_batch -= samp.p_batch
        self.buffer = Buffer(buffer_p_batch)
        self.batch_state = BatchState()
        self.log = Log()
        self.reward_decay = reward_decay
                
        all_cbs = [self.agent_cb] + [self.template_cb] + self.samplers + self.reward_cbs
        all_cbs += self.loss_cbs + cbs + [self.buffer] + [self.log]
        all_cbs = sorted(all_cbs, key=lambda x: x.order)
        
        self.register_cbs(all_cbs)
        self('setup')
        
    def __call__(self, event):
        for cb in self.cbs:
            cb(event)
        
    def register_cb(self, cb):
        if isinstance(cb, type): 
            cb = cb()
        cb.environment = self
        setattr(self, cb.name, cb)
        self.cbs.append(cb)
        
    def register_cbs(self, cbs):
        for cb in cbs:
            self.register_cb(cb)
            
    def remove_cb(self, cb):
        cb.environment = None
        cb.batch_state = None
        if hasattr(self, cb.name):
            delattr(self, cb.name)
            
        if cb in self.cbs:
            self.cbs.remove(cb)
        
    def remove_cbs(self, cbs):
        for cb in cbs:
            self.remove_cb(ccb)
        
    def build_buffer(self):
        start = time.time()
        if (len(self.buffer) < self.bs):
            self('build_buffer')
            self('after_build_buffer')
        end = time.time() - start
        self.log.timelog['build_buffer'].append(end)
            
    def sample_batch(self):
        start = time.time()
        self.batch_state = BatchState()
        for cb in self.cbs:
            cb.batch_state = self.batch_state
        self('before_batch') 
        self('sample_batch') 

        self('after_sample') 
        end = time.time() - start
        self.log.timelog['sample_batch'].append(end)
        
    def get_model_outputs(self):
        start = time.time()
        self('get_model_outputs')
        end = time.time() - start
        self.log.timelog['get_model_outputs'].append(end)
        
    def compute_reward(self):
        start = time.time()
        self('compute_reward')
        rewards = self.batch_state.rewards
        
        self.log.update_metric('rewards', rewards.mean().detach().cpu().numpy())
        
        self('after_compute_reward')
        end = time.time() - start
        self.log.timelog['compute_reward'].append(end)
        
    def compute_loss(self):
        start = time.time()
        self('compute_loss')
        loss = self.batch_state.loss
        self('zero_grad')
        try:
            loss.backward()
        except:
            # possibly no loss
            pass
        self('before_step')
        self('step')
        end = time.time() - start
        self.log.timelog['compute_loss'].append(end)
        
    def after_batch(self):
        start = time.time()
        self('after_batch')
        end = time.time() - start
        self.log.timelog['after_batch'].append(end)
        
    def fit(self, bs, sl, iters, report, cbs=[]):
        self.register_cbs(cbs)
        self.bs = bs
        self.sl = sl
        self.report = report
        mb = master_bar(range(1))
        self.log.pbar = mb
        self.log.report = report
        self('before_train')
        for _ in mb:
            for step in progress_bar(range(iters), parent=mb):
                self.build_buffer()
                self.sample_batch()
                self.get_model_outputs()
                self.compute_reward()
                self.compute_loss()
                self.after_batch()
                
        self('after_train')
        self.remove_cbs(cbs)
        

In [ ]:
# export

class Sampler(Callback):
    def __init__(self, name, buffer_size=0., p_batch=0., track=True):
        super().__init__()
        self.name = name
        self.buffer_size = buffer_size
        self.p_batch = p_batch
        self.track = track
            
    def _build_buffer(self):
        return []
        
    def _sample_batch(self):
        return []
        
    def build_buffer(self):
        outputs = self._build_buffer()
        if outputs:
            self.environment.buffer.add(outputs)
    
    def sample_batch(self):
        outputs = self._sample_batch()
        if outputs:
            self.batch_state.samples += outputs
            self.batch_state.sources += [self.name]*len(outputs)
                
                  
class ModelSampler(Sampler):
    def __init__(self, agent, model, name, buffer_size, p_batch, genbatch, latent=False, track=True,
                temperature=1., contrastive=False):
        super().__init__(name, buffer_size, p_batch, track)
        self.agent = agent
        self.model = model
        self.genbatch = genbatch
        self.latent = latent if self.agent.latents is not None else False
        self.temperature = temperature
        self.contrastive = contrastive
        
    def setup(self):
        if self.p_batch>0. and self.track:
            log = self.environment.log
            log.add_metric(f'{self.name}_diversity')
            log.add_metric(f'{self.name}_valid')
            log.add_metric(f'{self.name}_rewards')
            log.add_metric(f'{self.name}_new')
        
    def _build_buffer(self):
        env = self.environment
        bs = self.buffer_size
        outputs = []
        to_generate = bs
        
        if bs > 0:
            for batch in range(int(np.ceil(bs/self.genbatch))):
                current_bs = min(self.genbatch, to_generate)
                
                preds, _ = self.model.sample_no_grad(current_bs, env.sl, multinomial=True,
                                                     temperature=self.temperature)
                sequences = self.agent.reconstruct(preds)
                sequences = list(set(sequences))
                sequences = env.template_cb.filter_sequences(sequences)
                outputs += sequences
                outputs = list(set(outputs))
                to_generate = bs - len(outputs)
                
        return outputs
            
            
    def _sample_batch(self):
        env = self.environment
        bs = int(env.bs * self.p_batch)
        sequences = []
        
        if bs > 0:
            
            if self.latent:
                latents = self.agent.latents
                latent_idxs = torch.randint(0, latents.shape[0]-1, (bs,))
                sample_latents = latents[latent_idxs]
            else:
                sample_latents=None
            
            
            preds, _ = self.model.sample_no_grad(bs, env.sl, z=sample_latents, multinomial=True,
                                                temperature=self.temperature)
            sequences = self.agent.reconstruct(preds)

            env.batch_state[f'{self.name}_raw'] = sequences
            
            if sample_latents is not None:
                env.batch_state.latent_data[self.name] = latent_idxs
                
        return sequences
        
    def after_compute_reward(self):
        if self.p_batch>0. and self.track:
            state = self.environment.batch_state
            log = self.environment.log
            rewards = state.rewards.detach().cpu().numpy()
            sources = np.array(state.sources)
            
            if self.name in sources:
                log.update_metric(f'{self.name}_rewards', rewards[sources==self.name].mean())
            else:
                log.update_metric(f'{self.name}_rewards', 0.)
        
    def after_sample(self):
        if self.p_batch > 0. and self.track:
            log = self.environment.log
            state = self.environment.batch_state
            samples = state.samples
            sources = np.array(state.sources)==self.name

            samples = [samples[i] for i in range(len(samples)) if sources[i]]
            
            diversity = len(set(samples))/len(samples)
            valid = len(samples)/len(state[f'{self.name}_raw'])
            
            used = log.unique_samples
            novel = [i for i in samples if not i in used]
            percent_novel = len(novel)/len(samples)
            log.update_metric(f'{self.name}_new', percent_novel)
            log.update_metric(f"{self.name}_diversity", diversity)
            log.update_metric(f"{self.name}_valid", valid)

            
class ContrastiveSampler(Sampler):
    def __init__(self, base_sampler, agent, output_model, bs, repeats=1):
        super().__init__(base_sampler.name, base_sampler.buffer_size,
                         base_sampler.p_batch, base_sampler.track)
        
        self.base_sampler = base_sampler
        self.agent = agent
        self.output_model = output_model
        self.bs = bs
        self.repeats = repeats
        
    def __call__(self, event_name):
        
        event = getattr(self, event_name, None)
        
        if event is not None:
            output = event()
        else:
            output = None
            
        if not (event_name in ['build_buffer', 'sample_batch']):
            _ = self.base_sampler(event_name)
            
        return output
    
    def setup(self):
        self.base_sampler.environment = self.environment
        
    def sample_outputs(self, sequences):        
        env = self.environment
        if self.repeats>1:
            sequences = sequences*self.repeats
        pairs = [(i,'') for i in sequences]
        batch_ds = self.agent.dataset.new(pairs)
        
        if len(batch_ds)<self.bs:
        
            batch = batch_ds.collate_function([batch_ds[i] for i in range(len(batch_ds))])
            batch = to_device(batch)
            x,_ = batch
            z = self.output_model.x_to_latent(x)
            preds, _ = self.output_model.sample_no_grad(z.shape[0], env.sl, z=z)
            new_sequences = self.agent.reconstruct(preds)
            
        else:
            batch_dl = batch_ds.dataloader(self.bs, shuffle=False)

            new_sequences = []

            for i, batch in enumerate(batch_dl):
                batch = to_device(batch)
                x,_ = batch
                z = self.output_model.x_to_latent(x)
                preds, _ = self.output_model.sample_no_grad(z.shape[0], env.sl, z=z)
                new_sequences += self.agent.reconstruct(preds)
        
        outputs = [(sequences[i], new_sequences[i]) for i in range(len(sequences))]
        env.batch_state[f'{self.name}_raw_contrastive'] = outputs
                
        return outputs
        
    def _build_buffer(self):
        outputs = self.base_sampler._build_buffer()
        if outputs:
            outputs = self.sample_outputs(outputs)
        return outputs
    
    def _sample_batch(self):
        outputs = self.base_sampler._sample_batch()
        if outputs:
            outputs = self.sample_outputs(outputs)
        return outputs
    

In [ ]:
# export
        
class TemplateCallback(Callback):
    def __init__(self, template=None, weight=1., track=True, prefilter=True):
        super().__init__(order=-1)
        self.template = template
        self.track = track
        self.name = 'template'
        self.prefilter = prefilter
        self.weight = weight
        
    def setup(self):
        if self.track:
            log = self.environment.log
            log.add_metric(self.name)
            log.add_log(self.name)
        
    def compute_reward(self):
        env = self.environment
        state = env.batch_state
        
        if self.template is not None:
            rewards = np.array(self.template.eval_mols(state.samples))
        else:
            rewards = np.array([0.]*len(state.samples))
        
        hps = self.get_hps(state.samples)
        state[self.name] = rewards
        
        if self.track:
            env.log.update_metric(self.name, rewards.mean())
            
        state.template_passes = hps
        state.rewards += to_device(torch.from_numpy(self.weight*rewards).float())
        
    def get_hps(self, sequences):
        if self.template is not None:
            hps = np.array(self.template(sequences))
        else:
            hps = np.array([True]*len(sequences))
            
        return hps
        
    def filter_sequences(self, sequences, return_array=False):
        if self.prefilter:
            passes = self.get_hps(sequences)
        else:
            passes = self.validate(sequences)
        
        if return_array:
            output = passes
        else:
            output  = [sequences[i] for i in range(len(sequences)) if passes[i]]
        return output
    
    def standardize(self, sequences):
        if self.template is not None:
            sequences = self.template.standardize(sequences)
        
        return sequences
    
    def validate(self, sequences):
        if self.template is not None:
            valid = np.array(self.template.validate(sequences))
        else:
            valid = np.array([True]*len(sequences))
            
        return valid

    
class ContrastiveTemplate(TemplateCallback):
    def __init__(self, similarity_function, max_score=None, template=None, 
                 weight=1., track=True, prefilter=True):
        super().__init__(template=template, weight=weight, track=track, prefilter=prefilter)
        self.similarity_function = similarity_function
        self.max_score = max_score
        
    def setup(self):
        if self.track:
            log = self.environment.log
            log.add_metric(self.name)
            log.add_metric(self.name+'_temp')
            log.add_metric(self.name+'_sim')
            log.add_log(self.name)
            log.add_log(self.name+'_temp')
            log.add_log(self.name+'_sim')
        
    def compute_reward(self):
        env = self.environment
        state = env.batch_state
        
        samples = state.samples
        source_samples = [i[0] for i in samples]
        target_samples = [i[1] for i in samples]
        hps = self.get_hps(samples)
        
        if template is not None:
            source_rewards = np.array(self.template.eval_mols(source_samples))
            target_rewards = np.array(self.template.eval_mols(target_samples))
            
            rewards = target_rewards - source_rewards
            if self.max_score is not None:
                rewards = rewards / (self.max_score-source_rewards)
                
            sim_scores = self.similarity_function.score(source_samples, target_samples)
            rewards = rewards
            
        else:
            rewards = np.array([0.]*len(state.samples))
            sim_scores = np.array([0.]*len(state.samples))
        
        state.template = rewards
        state.template_sim = sim_scores
        
        full_rewards = rewards + sim_scores
        
        if self.track:
            env.log.update_metric(self.name, full_rewards.mean())
            env.log.update_metric(self.name+'_temp', rewards.mean())
            env.log.update_metric(self.name+'_sim', sim_scores.mean())
            
        state[self.name] = full_rewards
        state[self.name+'_temp'] = rewards
        state[self.name+'_sim'] = sim_scores
            
        state.template_passes = hps
        state.rewards += to_device(torch.from_numpy(self.weight*full_rewards).float())
        
    def standardize(self, sequences):
        if self.template is not None:
            sources = self.template.standardize([i[0] for i in sequences])
            targets = self.template.standardize([i[1] for i in sequences])
            sequences = [(sources[i], targets[i]) for i in range(len(sources))]
        
        return sequences
        
    def get_hps(self, sequences):
        
        if type(sequences[0])==str:
            hps = super().get_hps(sequences)
        else:
            source_sequences = [i[0] for i in sequences]
            target_sequences = [i[1] for i in sequences]
            s_hps = super().get_hps(source_sequences)
            t_hps = super().get_hps(target_sequences)
            sim_bools = self.similarity_function.bools(source_sequences, target_sequences)
            hps = s_hps*t_hps*sim_bools
            
        return hps
    
    def validate(self, sequences):
        if type(sequences[0])==str:
            valid = super().validate(sequences)
        else:
            s_val = super().validate([i[0] for i in sequences])
            t_val = super().validate([i[1] for i in sequences])
            valid = s_val*t_val
            
        return valid
        

In [ ]:
# export

class AgentCallback(Callback):
    def __init__(self, agent, name, clip=1.):
        super().__init__(order=20)
        self.agent = agent
        self.name = name
        self.clip = clip
        
    def zero_grad(self):
        self.agent.zero_grad()
    
    def before_step(self):
        nn.utils.clip_grad_norm_(self.agent.model.parameters(), self.clip)
        
    def step(self):
        self.agent.step()
        
    def after_sample(self):
        env = self.environment
        sequences = self.batch_state.samples
        diversity = len(set(sequences))/len(sequences)
                
        bs = len(sequences)
        self.batch_state.rewards = to_device(torch.zeros(bs))
        
    def get_model_outputs(self):
        # get relevant model outputs
        pass
        
class GenAgentCallback(AgentCallback):
    def __init__(self, agent, name, contrastive=False):
        super().__init__(agent, name)
        self.contrastive = contrastive
    
    def after_sample(self):
        env = self.environment
        sequences = self.batch_state.samples
                
        batch_ds = self.agent.dataset.new(sequences)
        batch = batch_ds.collate_function([batch_ds[i] for i in range(len(batch_ds))])
        batch = to_device(batch)
        bs = len(batch_ds)
        x,y = batch
            
        self.batch_state.x = x
        self.batch_state.y = y
        self.batch_state.bs = bs
        mask = ~(y==self.agent.vocab.stoi['pad'])
        self.batch_state.mask = mask
        self.batch_state.lengths = mask.sum(-1)
        self.batch_state.sl = y.shape[-1]
        self.batch_state.rewards = to_device(torch.zeros(bs))
        self.batch_state.trajectory_rewards = to_device(torch.zeros(y.shape))
        
    def subset_tensor(self, x, mask):
        if type(x)==list:
            x = [i[mask] for i in x]
        else:
            x = x[mask]
        
        return x
    
    def get_rl_tensors(self, model, x, y, latent_info, sources):
        if latent_info:
            latent_sources = []
            output_tensors = []
            for (latent_source, latent_idxs) in latent_info.items():
                latent_sources.append(latent_source)
                latent_mask = torch.tensor([i==latent_source for i in sources]).bool()
                latents = self.agent.latents[latent_idxs]
                out = self.agent.model.get_rl_tensors(self.subset_tensor(x, latent_mask), 
                                                      self.subset_tensor(y, latent_mask),
                                                      latent=latents)
                output_tensors.append(out)
                
            non_latent_mask = torch.tensor([not i in latent_sources for i in sources]).bool()
            if non_latent_mask.sum()>0:
                out = model.get_rl_tensors(self.subset_tensor(x, non_latent_mask), 
                                                      self.subset_tensor(y, non_latent_mask))
                output_tensors.append(out)
            
            mo = torch.cat([i[0] for i in output_tensors], 0)
            mlp = torch.cat([i[1] for i in output_tensors], 0)
            mglp = torch.cat([i[2] for i in output_tensors], 0)
            me = torch.cat([i[3] for i in output_tensors], 0)
            
        else:
            mo, mlp, mglp, me = model.get_rl_tensors(x,y)
            
        return mo, mlp, mglp, me 
        
    def get_model_outputs(self):
            
        x = self.batch_state.x
        y = self.batch_state.y
        sources = self.batch_state.sources
        latent_info = self.batch_state.latent_data
            
        mo, mlp, mglp, me = self.get_rl_tensors(self.agent.model, x, y, latent_info, sources)
        mprob = mlp.exp()
        
        self.batch_state.model_output = mo
        self.batch_state.model_logprobs = mlp
        self.batch_state.model_gathered_logprobs = mglp
        self.batch_state.model_encoded = me
        self.batch_state.y_gumbel = F.one_hot(y, len(self.agent.vocab.itos)) + mprob - mprob.detach()
        
        if self.agent.value_head is not None:
            value_predictions = self.agent.value_head(me)
            with torch.no_grad():
                base_value_predictions = self.agent.base_value_head(me)
        else:
            value_predictions = None
            base_value_predictions = None
            
        self.batch_state.state_values = value_predictions
        self.batch_state.ref_state_values = base_value_predictions
        
        if self.agent.base_model is not None:
            with torch.no_grad():
#                 bo, blp, bglp, be = self.agent.base_model.get_rl_tensors(x,y)
                bo, blp, bglp, be = self.get_rl_tensors(self.agent.base_model, x, y, latent_info, sources)    
        else:
            bo, blp, bglp, be = None, None, None, None
            
        self.batch_state.reference_output = bo
        self.batch_state.reference_logprobs = blp
        self.batch_state.reference_gathered_logprobs = bglp
        self.batch_state.reference_encoded = be
        


In [ ]:
# export

class RewardCallback(Callback):
    def __init__(self, reward_function, name, weight=1., track=True):
        super().__init__(order=1)
        self.name = name
        self.reward_function = reward_function
        self.weight = weight
        self.track = track
        
    def setup(self):
        if self.track:
            log = self.environment.log
            log.add_metric(self.name)
            log.add_log(self.name)
        
    def compute_reward(self):
        rewards, reward_dict = self.reward_function.from_batch_state(self.batch_state)
        
        if self.track:
            self.environment.log.update_metric(self.name, rewards.mean().detach().cpu().numpy())
            
        rewards = rewards * self.weight
        self.batch_state.rewards += rewards
        self.batch_state[self.name] = reward_dict

class LossCallback(Callback):
    def __init__(self, loss_function, name, weight=1., track=True):
        super().__init__(order=1)
        self.name = name
        self.loss_function = loss_function
        self.weight = weight
        self.track = track
        
    def setup(self):
        if self.track:
            log = self.environment.log
            log.add_metric(self.name)
            log.add_log(self.name)
        
    def compute_loss(self):
        loss, loss_dict = self.loss_function.from_batch_state(self.batch_state)
        
        if self.track:
            self.environment.log.update_metric(self.name, loss.mean().detach().cpu().numpy())
            
        loss = loss * self.weight
        self.batch_state.loss += loss
        self.batch_state[self.name] = loss_dict
        
        
class PriorLoss():
    def __init__(self, agent):
        self.agent = agent
        
    def pg_loss(self, x, rewards):
        model = self.agent.model
        prior = self.agent.model.prior

        old_prior = self.agent.base_model.prior
        
        z = model.x_to_latent(x)
        prior_lps = prior.log_prob(z)

        with torch.no_grad():
            old_lps = old_prior.log_prob(z)

        ratios = prior_lps - old_lps.detach()

        rewards = rewards - rewards.mean()

        prior_loss = (-ratios.sum(-1)*rewards)
        prior_loss = torch.clip(prior_loss, -256, 256)
        prior_loss = prior_loss.mean()
        
        return prior_loss
    
    def from_batch_state(self, batch_state):
        x = batch_state.x
        rewards = batch_state.rewards
        prior_loss = self.pg_loss(x, rewards)
        return prior_loss, {}
    
class HistoricPriorLoss(Callback):
    def __init__(self, agent, percentile, n, start_iter):
        super().__init__(name='prior_his', order=10)
        self.agent = agent
        self.loss = PriorLoss(agent)
        self.percentile = percentile
        self.n = n
        self.start_iter = start_iter
        
    def setup(self):
        log = self.environment.log
        log.add_metric(self.name)
        
    def compute_loss(self):
        loss = self.historic_loss()
        
        self.batch_state.loss += loss
        self.environment.log.update_metric(self.name, loss.detach().cpu().numpy())
        
    def historic_loss(self):
        env = self.environment
        
        iterations = self.environment.log.iterations

        if iterations > self.start_iter:
            df = log_to_df(env.log.log, ['samples', 'rewards'])
            df.drop_duplicates(subset='samples', inplace=True)
            
            df1 = df[df.rewards>np.percentile(df.rewards.values, self.percentile)]
            n_samp = min(self.n, df1.shape[0])
            samples1 = df1.sample(n=n_samp)
            
            df2 = df[df.rewards<np.percentile(df.rewards.values, self.percentile)]
            n_samp = min(self.n, df2.shape[0])
            samples2 = df2.sample(n=n_samp)
            
            df = pd.concat([samples1, samples2])
            
            rewards = to_device(torch.tensor(df.rewards.values).float())
            
            
            batch_ds = self.agent.dataset.new(df.samples.values)
            batch = batch_ds.collate_function([batch_ds[i] for i in range(len(batch_ds))])
            batch = to_device(batch)
            x,y = batch
            
            prior_loss = self.loss.pg_loss(x, rewards)

        else:
            prior_loss = torch.tensor(0.)
        
        return prior_loss

In [ ]:
# export

class UpdateBaselineCB(Callback):
    def __init__(self, agent, iters, name):
        super().__init__(order=10, name=name)
        self.agent = agent
        self.iters = iters
        self.num_updates = 0
        
    def after_batch(self):
        log = self.environment.log
        iterations = log.iterations
        if iterations%self.iters == 0 and iterations>0:
            self.agent.update_base_models()
            self.num_updates += 1
            
            
class StatsCallback(Callback):
    def __init__(self, grabname, name, order):
        self.grabname = grabname
        self.name = name
        self.order = order
        
    def setup(self):
        log = self.environment.log
        log.add_metric(f'{self.grabname}_p90')
        log.add_metric(f'{self.grabname}_max')
        
    def after_compute_reward(self):
        log = self.environment.log
        state = self.environment.batch_state
        rewards = state.rewards.detach().cpu().numpy()
        sources = np.array(state.sources)
        
        rewards = rewards[sources==self.grabname]
        
        log.update_metric(f'{self.grabname}_p90', np.percentile(rewards, 90))
        log.update_metric(f'{self.grabname}_max', rewards.max())

class Rollback(Callback):
    def __init__(self, model1, model2, metric, lookback, target, alpha, name):
        super().__init__(order=10, name=name)
        self.model1 = model1
        self.model2 = model2
        self.metric = metric
        self.lookback = lookback
        self.target = target
        self.alpha = alpha
        self.last_rollback = 0
        
    def after_batch(self):
        env = self.environment
        log = env.log
        current_val = np.array(log.metrics[self.metric][-self.lookback:]).mean()
    
        if current_val < self.target and self.last_rollback <= 0:
            print('rollback')
            merge_models(self.model1, self.model2, alpha=self.alpha)
            self.last_rollback = self.lookback
            
        self.last_rollback -= 1
        
class RetrainRollback(Callback):
    def __init__(self, agent, percentile, base_update, lr, bs, metric, lookback, target, name):
        super().__init__(order=1000, name=name)
        self.agent = agent
        self.percentile = percentile
        self.base_update = base_update
        self.lr = lr
        self.bs = bs
        self.metric = metric
        self.lookback = lookback
        self.target = target
        self.last_rollback = 0
        
    def after_batch(self):
        env = self.environment
        log = env.log
        current_val = np.array(log.metrics[self.metric][-self.lookback:]).mean()
    
        if current_val < self.target and self.last_rollback <= 0:
            self.train_model()
            self.last_rollback = self.lookback
            
        self.last_rollback -= 1
        
    def train_model(self):
        env = self.environment
        df = log_to_df(env.log.log, ['samples', 'rewards'])
        df.drop_duplicates(subset='samples', inplace=True)
        df = df[df.rewards>np.percentile(df.rewards.values, self.percentile)]

        self.agent.update_dataset_from_inputs(df.samples.values)
        self.agent.train_supervised(self.bs, 1, self.lr)

        merge_models(self.agent.base_model, self.agent.model, alpha=self.base_update)     
        
class SupevisedCB(Callback):
    def __init__(self, agent, frequency, base_update, percentile, lr, bs):
        super().__init__('supervised', order=1000)
        self.agent = agent
        self.frequency = frequency
        self.base_update = base_update
        self.percentile = percentile
        self.lr = lr
        self.bs = bs
        
    def after_batch(self):
        env = self.environment
        iterations = self.environment.log.iterations
        
        if iterations>0 and iterations%self.frequency==0:
            self.train_model()
            
            
    def train_model(self):
        env = self.environment
        df = log_to_df(env.log.log, ['samples', 'rewards'])
        df.drop_duplicates(subset='samples', inplace=True)
        df = df[df.rewards>np.percentile(df.rewards.values, self.percentile)]

        self.agent.update_dataset_from_inputs(df.samples.values)
        self.agent.train_supervised(self.bs, 1, self.lr)

        merge_models(self.agent.base_model, self.agent.model, alpha=self.base_update)      


class LogSampler(Sampler):
    def __init__(self, sample_name, start_iter, percentile, buffer_size):
        super().__init__(sample_name+'_sample', buffer_size, p_batch=0.)
        self.start_iter = start_iter
        self.percentile = percentile
        self.sample_name = sample_name
        
    def _build_buffer(self):
        env = self.environment
        
        iterations = self.environment.log.iterations
        outputs = []

        if iterations > self.start_iter:
            df = log_to_df(env.log.log, ['samples', self.sample_name])
            df.drop_duplicates(subset='samples', inplace=True)
            bs = self.buffer_size
            if bs > 0:
                
                subset = df[df[self.sample_name]>np.percentile(df[self.sample_name].values, 
                                                               self.percentile)]
                outputs = list(subset.sample(n=min(bs, subset.shape[0])).samples.values)
        
        return outputs
                    
class LogEnumerator(Sampler):
    def __init__(self, sample_name, start_iter, buffer_size, percentile):
        super().__init__(sample_name+'_enum', buffer_size, p_batch=0.)
        self.start_iter = start_iter
        self.atom_types = ['C', 'N', 'O', 'F', 'S', 'Br', 'Cl', -1]
        self.sample_name = sample_name
        self.percentile = percentile
        
    def _build_buffer(self):
        
        env = self.environment
        
        iterations = self.environment.log.iterations
        outputs = []

        if iterations > self.start_iter:
            df = log_to_df(env.log.log, ['samples', self.sample_name])
            df.drop_duplicates(subset='samples', inplace=True)
            bs = self.buffer_size
                
            subset = df[df[self.sample_name]>np.percentile(df[self.sample_name].values, 
                                                           self.percentile)]
            samples = list(subset.sample(n=min(bs, subset.shape[0])).samples.values)
            outputs = []

            for s in samples:
                new_smiles = add_atom_combi(s, self.atom_types) + add_bond_combi(s)
                new_smiles = [i for i in new_smiles if i is not None]
                new_smiles = [i for i in new_smiles if not '.' in i]
                outputs += new_smiles

        return outputs
                    
class DatasetSampler(Sampler):
    def __init__(self, data, buffer_size, name):
        super().__init__(name, buffer_size, 0.)
        self.data = data
        
    def _build_buffer(self):
        idxs = np.random.randint(0, len(self.data), self.buffer_size)
        samples = [self.data[i] for i in idxs]
        return samples
        
# class NoveltyBonus(Callback):
#     def __init__(self, name, reward):
#         super().__init__(name=name, order=1)
#         self.reward = reward
        
#     def setup(self):
#         log = self.environment.log
#         log.add_metric(self.name)
#         log.add_log(self.name)
        
#     def compute_reward(self):
        
#         log = self.environment.log
#         state = self.environment.batch_state
#         samples = np.array(state.samples)
        
#         used = log.unique_samples
        
#         new = [not i in used for i in samples]
#         reward = np.array([self.reward*i for i in new])
#         self.batch_state.rewards += to_device(torch.tensor(reward).float())
#         self.batch_state[self.name] = reward

#         log.update_metric(self.name, reward.mean())

def log_to_df(log, keys=None):
    batch = 0
    output_dict = defaultdict(list)
    
    if keys is None:
        keys = list(log.keys())
    
    items = log[keys[0]]
    for item in items:
        output_dict['batch'] += [batch]*len(item)
        batch += 1
        
    for key in keys:
        output_dict[key] = flatten_list_of_lists(log[key])
        
    return pd.DataFrame(output_dict)